In [21]:
import numpy as np
import plotly.graph_objects as go
from scipy.stats import linregress

In [22]:

file_path = "3d_shape_points_data.npz"
data = np.load(file_path)
data


NpzFile '3d_shape_points_data.npz' with keys: points

In [23]:
points = data["points"]  # Assuming points are stored under the key "points"

# Extract the points data
points = data["points"]

# Get the shape of the points array and preview the first few entries
points_shape = points.shape
points_sample = points[:5]  # First 5 points

points_shape, points_sample

((65536, 3),
 array([[9.998086  , 0.        , 0.        ],
        [9.994803  , 0.24535865, 0.        ],
        [9.985485  , 0.4905554 , 0.        ],
        [9.970136  , 0.7354414 , 0.        ],
        [9.948765  , 0.9798679 , 0.        ]], dtype=float32))

In [24]:


# Load the point cloud data
file_path = "3d_shape_points_data.npz"
data = np.load(file_path)
points = data["points"]

# Create a 3D scatter plot with Plotly
fig = go.Figure()
fig.add_trace(go.Scatter3d(
    x=points[:, 0], 
    y=points[:, 1], 
    z=points[:, 2], 
    mode='markers',
    marker=dict(size=1, opacity=0.5)
))

# Set axis labels
fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ),
    title="3D Point Cloud of Deformed Component"
)

fig.show()


In [25]:

# Extract coordinates
x, y, z = points[:, 0], points[:, 1], points[:, 2]
radii = np.sqrt(x**2 + y**2)  # Compute radius

# Analyze radius variation along height
num_bins = 50
bins = np.linspace(np.min(z), np.max(z), num_bins)

# Compute average radii for each height bin
avg_radii = [np.mean(radii[(z >= bins[i]) & (z < bins[i+1])]) for i in range(num_bins-1)]

# Fit a line to estimate frustum shape (radius vs height)
slope, intercept, _, _, _ = linregress(bins[1:], avg_radii)

# Compute corrected radii for each height level
r_corrected = slope * z + intercept

# Convert to cylindrical coordinates for correction
theta = np.arctan2(y, x)
x_corrected = r_corrected * np.cos(theta)
y_corrected = r_corrected * np.sin(theta)

# Create 3D scatter plots using Plotly (Comparison of Original vs. Reconstructed)
fig = go.Figure()
fig.add_trace(go.Scatter3d(x=points[:, 0], y=points[:, 1], z=points[:, 2],
                           mode='markers', marker=dict(size=2, color='red'),
                           name='Original Deformed Shape'))
fig.add_trace(go.Scatter3d(x=x_corrected, y=y_corrected, z=z,
                           mode='markers', marker=dict(size=2, color='blue'),
                           name='Reconstructed Frustum'))

fig.update_layout(title='3D Shape Reconstruction',
                  scene=dict(xaxis_title='X',
                             yaxis_title='Y',
                             zaxis_title='Z'))

fig.show()
